In [116]:
import os
import numpy as np
import pandas as pd
import scipy.sparse
from scipy.spatial.distance import correlation

In [165]:
data=pd.read_csv('clean_travel_data.csv')
data.head()

,location,place,state,location_rating,age,season,affordable,types_of_travel,religion,itemId
0,Bhoramdeo Temple,"Chaura village, Kawardha town",chhattisgarh,4.0,0,winter,Yes,family,hindu,1
1,maitri bagh,"Near to Maroda, Bhilai",chhattisgarh,4.0,20,"summer,spring,winter,rainy,autumn",Yes,friends,any,2
2,Tirathgarh Falls,Bastar district,chhattisgarh,4.6,25,monsoon,Yes,friends,any,3
3,Sirpur Heritage Site,bank of holy Mahanadi river,chhattisgarh,5.0,0,winter,Yes,family,hindu,4
4,Rajeev Smriti Van,Raipur,chhattisgarh,3.9,0,"summer,spring,winter,rainy,autumn",Yes,solo,any,5


In [166]:
data1=pd.read_csv('collaborative.csv')
data1.head()

,userId,itemId,rating,timestamp
0,1,1,NaN,881250949
1,1,2,3.0,891717742
2,1,3,1.0,878887116
3,1,4,NaN,880606923
4,1,5,1.0,886397596


In [167]:
data=pd.merge(data,data1,left_on='itemId',right_on="itemId")
data.head()

,location,place,state,location_rating,age,season,affordable,types_of_travel,religion,itemId,userId,rating,timestamp
0,Bhoramdeo Temple,"Chaura village, Kawardha town",chhattisgarh,4.0,0,winter,Yes,family,hindu,1,1,NaN,881250949
1,Bhoramdeo Temple,"Chaura village, Kawardha town",chhattisgarh,4.0,0,winter,Yes,family,hindu,1,2,3.0,891717742
2,Bhoramdeo Temple,"Chaura village, Kawardha town",chhattisgarh,4.0,0,winter,Yes,family,hindu,1,3,1.0,878887116
3,Bhoramdeo Temple,"Chaura village, Kawardha town",chhattisgarh,4.0,0,winter,Yes,family,hindu,1,4,NaN,880606923
4,Bhoramdeo Temple,"Chaura village, Kawardha town",chhattisgarh,4.0,0,winter,Yes,family,hindu,1,5,1.0,886397596


In [168]:
data=pd.DataFrame.sort_values(data,['userId','itemId'])
data

,location,place,state,location_rating,age,season,affordable,types_of_travel,religion,itemId,userId,rating,timestamp
0,Bhoramdeo Temple,"Chaura village, Kawardha town",chhattisgarh,4.0,0,winter,Yes,family,hindu,1,1,NaN,881250949
10,maitri bagh,"Near to Maroda, Bhilai",chhattisgarh,4.0,20,"summer,spring,winter,rainy,autumn",Yes,friends,any,2,1,3.0,891717742
20,Tirathgarh Falls,Bastar district,chhattisgarh,4.6,25,monsoon,Yes,friends,any,3,1,1.0,878887116
30,Sirpur Heritage Site,bank of holy Mahanadi river,chhattisgarh,5.0,0,winter,Yes,family,hindu,4,1,NaN,880606923
40,Rajeev Smriti Van,Raipur,chhattisgarh,3.9,0,"summer,spring,winter,rainy,autumn",Yes,solo,any,5,1,1.0,886397596
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2819,Jal Mandir,Pawapuri,Bihar,4.1,30,monsoon,Yes,"solo,family,friends",jains,284,10,NaN,891352093
2829,Hajipur,Hajipur,Bihar,3.9,25,"summer,spring,winter,rainy,autumn",Yes,"friends,family,solo","hindus,muslims,buddhists,jains",285,10,4.0,886122952
2839,Karkat Waterfall,Karkathgarh,Bihar,4.5,20,monsoon,Yes,friends,any,286,10,1.0,878961814
2849,Dhungeswari hills,Larpur,Bihar,4.0,20,"summer,spring,winter,rainy,autumn",Yes,"friends,family",buddhists,287,10,1.0,884140579


In [169]:
userItemRatingMatrix=pd.pivot_table(data, values='rating',index=['userId'], columns=['itemId'])
userItemRatingMatrix

itemId,1,2,3,4,5,6,7,8,9,10,...,279,280,281,282,283,284,285,286,287,288
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,3.0,1.0,NaN,1.0,NaN,NaN,5.0,3.0,NaN,...,NaN,NaN,4.0,1.0,1.0,NaN,3.0,NaN,4.0,NaN
2,3.0,1.0,NaN,1.0,NaN,NaN,5.0,3.0,NaN,2.0,...,NaN,4.0,1.0,1.0,NaN,3.0,NaN,4.0,NaN,NaN
3,1.0,NaN,1.0,NaN,NaN,5.0,3.0,NaN,2.0,NaN,...,4.0,1.0,1.0,NaN,3.0,NaN,4.0,NaN,NaN,3.0
4,NaN,1.0,NaN,NaN,5.0,3.0,NaN,2.0,NaN,5.0,...,1.0,1.0,NaN,3.0,NaN,4.0,NaN,NaN,3.0,1.0
5,1.0,NaN,NaN,5.0,3.0,NaN,2.0,NaN,5.0,3.0,...,1.0,NaN,3.0,NaN,4.0,NaN,NaN,3.0,1.0,NaN
6,NaN,NaN,5.0,3.0,NaN,2.0,NaN,5.0,3.0,NaN,...,NaN,3.0,NaN,4.0,NaN,NaN,3.0,1.0,NaN,1.0
7,NaN,5.0,3.0,NaN,2.0,NaN,5.0,3.0,NaN,3.0,...,3.0,NaN,4.0,NaN,NaN,3.0,1.0,NaN,1.0,NaN
8,5.0,3.0,NaN,2.0,NaN,5.0,3.0,NaN,3.0,NaN,...,NaN,4.0,NaN,NaN,3.0,1.0,NaN,1.0,NaN,NaN
9,3.0,NaN,2.0,NaN,5.0,3.0,NaN,3.0,NaN,2.0,...,4.0,NaN,NaN,3.0,1.0,NaN,1.0,NaN,NaN,5.0


In [170]:
def similarity(user1,user2):
    try:
        user1=np.array(user1)-np.nanmean(user1)
        user2=np.array(user2)-np.nanmean(user2)
        commonItemIds=[i for i in range(len(user1)) if user1[i]>0 and user2[i]>0]
        if len(commonItemIds)==0:
           return 0
        else:
           user1=np.array([user1[i] for i in commonItemIds])
           user2=np.array([user2[i] for i in commonItemIds])
           return correlation(user1,user2)
    except ZeroDivisionError:
        print("You can't divide by zero!")

In [171]:
def nearestNeighbourRatings(activeUser,K):
    try:
        similarityMatrix=pd.DataFrame(index=userItemRatingMatrix.index,columns=['Similarity'])
        for i in userItemRatingMatrix.index:
            similarityMatrix.loc[i]=similarity(userItemRatingMatrix.loc[activeUser],userItemRatingMatrix.loc[i])
        similarityMatrix=pd.DataFrame.sort_values(similarityMatrix,['Similarity'],ascending=[0])
        nearestNeighbours=similarityMatrix[:K]
        neighbourItemRatings=userItemRatingMatrix.loc[nearestNeighbours.index]
        predictItemRating=pd.DataFrame(index=userItemRatingMatrix.columns, columns=['Rating'])
        for i in userItemRatingMatrix.columns:
            predictedRating=np.nanmean(userItemRatingMatrix.loc[activeUser])
            for j in neighbourItemRatings.index:
                if userItemRatingMatrix.loc[j,i]>0:
                   predictedRating += (userItemRatingMatrix.loc[j,i]-np.nanmean(userItemRatingMatrix.loc[j]))*nearestNeighbours.loc[j,'Similarity']
                predictItemRating.loc[i,'Rating']=predictedRating
    except ZeroDivisionError:
        print("You can't divide by zero!")            
    return predictItemRating

In [172]:
def topNRecommendations(activeUser,N):
    try:
        predictItemRating=nearestNeighbourRatings(activeUser,N)
        placeAlreadyWatched=list(userItemRatingMatrix.loc[activeUser].loc[userItemRatingMatrix.loc[activeUser]>0].index)
        predictItemRating=predictItemRating.drop(placeAlreadyWatched)
        topRecommendations=pd.DataFrame.sort_values(predictItemRating,['Rating'],ascending=[0])[:N]
        topRecommendationTitles=(data.loc[data1.itemId.isin(topRecommendations.index)])
    except ZeroDivisionError:
        print("You can't divide by zero!")
    return list([topRecommendationTitles.location,topRecommendationTitles.place,topRecommendationTitles.state,topRecommendationTitles.location_rating])

In [173]:
activeUser=int(input("Enter userid: "))

Enter userid: 10


In [174]:
print("The recommended places for you are: ")
topN=pd.DataFrame(topNRecommendations(activeUser,4))
topN=topN.T
topN=topN.sort_values(by='location_rating', ascending=False)
topN

The recommended places for you are: 


,location,place,state,location_rating
52,Palolem Beach,Canacona,Goa,5
2391,SHAR,Srihari Kota,Andhra Pradesh,4.7
2418,Yaganti Temple,Yaganti,Andhra Pradesh,4.6
106,dudhsagar-falls,�Mandovi River�,Goa,4.6
630,mysore,mysore,Karnataka,4.5
973,Pench National Park,Nagpur,Madhya Pradesh,4.5
2648,Tocklai Tea Research Centre,Jorhat,Assam,4.5
368,Manikaran Gurudwara,Kullu,Himachal pradesh,4.5
657,hampi,Vijayanagara,Karnataka,4.5
1786,golden beach,puri,odisha,4.5


In [175]:
def favoritePlace(activeUser,N):
    topPlace=pd.DataFrame.sort_values(data[data.userId==activeUser],['rating'],ascending=[0])[:N]
    return list([topPlace.location,topPlace.place,topPlace.state,topPlace.location_rating])

In [176]:
print("The recommended places for you are: ")
fav_place=pd.DataFrame(favoritePlace(activeUser,4))
fav_place=fav_place.T
fav_place=fav_place.sort_values(by='location_rating', ascending=False)
fav_place

The recommended places for you are: 


,location,place,state,location_rating
489,Vaishnavo Devi Mandir,Trikuta hills,Jammu & Kashmir,4.5
689,gokarna,uttar kannada,Karnataka,4.5
1259,RKCS Art Gallery And Museum,Imphal,Manipur,4.4
579,Jamshedpur,Jamshedpur,Jharkhand,4
